# OpenAI Function Calling 101
One of the struggles of using LLMs like ChatGPT is that they do not produce a structured data output. This is important for programmatic systems that largely rely on structured data for system interaction. For example, if you want to build a program that analyzes the sentiment of a movie review, you might have to execute a prompt that looks like the following:

```
prompt = f'''
Please perform a sentiment analysis on the following movie review:
{MOVIE_REVIEW_TEXT}
Please output your response as a single word: either "Positive" or "Negative". Do not add any extra characters.
'''
```

The problem with this is that it doesn't always work. It's pretty common that the LLM will throw in an undesired period or longer explanation like: "The sentiment of this movie is: Positive." While you can regex out the answer (🤢), this is obviously not ideal. What would be ideal is if the LLM would return the output as something like the following structured JSON:

```
{
    'sentiment': 'positive'
}
```

Enter **OpenAI's new function calling**! Function calling is precisely the answer to the problem above. This Jupyter notebook will demonstrate a simple example of how to use OpenAI's new function calling in Python. If you would like to see the full documentation, [please check out this link](https://platform.openai.com/docs/guides/gpt/function-calling).

## Notebook Setup
Let's start with our imports. Now, you may already have the `openai` Python client already installed, but you'll most likely need to upgrade it to get the new function calling functionality. Here's how to do this upgrade in your Terminal / Powershell with `pip`:

```
pip install openai --upgrade
```

In [1]:
# Importing the necessary Python libraries
import json
import yaml
import openai

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Applying our API key and organization ID to OpenAI
openai.organization = keys_yaml['ORG_ID']
openai.api_key = keys_yaml['API_KEY']

To test out the function calling functionality, I wrote a short "About Me" containing particular facts that we'll be parsing out into appropriate data structures, including integers and strings. Let's load in this text

In [3]:
# Loading the "About Me" text from local file
with open('../data/about-me.txt', 'r') as f:
    about_me = f.read()

print(about_me)

Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda.


## The Pre-Function Calling Days
Before we demonstrate function calling, let's demonstrate how we used to use prompt engineering and Regex to produce a structure JSON that we can programmatically work with down the road.

In [4]:
# Engineering a prompt to extract as much information from "About Me" as a JSON object
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information.
Name
Job title
Company
Number of children as a single number
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [5]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

In [6]:
# Loading the response as a JSON object
json_response = json.loads(openai_response['choices'][0]['message']['content'])
json_response

{'name': 'David Hundley',
 'job title': 'Principal Machine Learning Engineer',
 'company': 'State Farm',
 'number of children': 2,
 'car make': 'Tesla',
 'car model': 'Model 3',
 'favorite video game series': 'The Legend of Zelda'}

## Using the New Function Calling Capabilities
Now that we've demonstrated how we used to get structured JSON in the pre-function calling days, let's move into how we can now make use of function calling to extract the same results but in a more consistent manner for our systematic usage.

In [57]:
# Defining how we want ChatGPT to call our custom functions
functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_car_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

In [47]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}],
    functions = functions,
    function_call = 'auto'
)

json.loads(openai_response['choices'][0]['message']['function_call']['arguments'])

{'name': 'David Hundley',
 'job_title': 'Principal Machine Learning Engineer',
 'company_name': 'State Farm',
 'num_children': 2,
 'vehicle_make': 'Tesla',
 'vehicle_model': 'Model 3',
 'favorite_vg_series': 'The Legend of Zelda'}

In [41]:
type(json.loads(openai_response['choices'][0]['message']['function_call']['arguments'])['num_children'])

int

In [33]:
openai_response['choices'][0]['message']['function_call']['arguments']

'{\n  "name": "David Hundley",\n  "job_title": "principal machine learning engineer",\n  "num_children": 2\n}'

In [56]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {'role': 'system', 'content': 'If the name is unknown, do not perform function calling.'},
        {'role': 'user', 'content': 'I am a principal machine learning engineer with 3 children.'}
    ],
    functions = functions,
    function_call = 'auto'
)

openai_response

<OpenAIObject chat.completion id=chatcmpl-7a18Cw5JcATrDD64wM81zVthjG36Z at 0x127480770> JSON: {
  "id": "chatcmpl-7a18Cw5JcATrDD64wM81zVthjG36Z",
  "object": "chat.completion",
  "created": 1688818512,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_person_info",
          "arguments": "{\n  \"name\": \"unknown\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 3\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 285,
    "completion_tokens": 35,
    "total_tokens": 320
  }
}